<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Kafka Consumer 

**Technical Accomplishments:**
- Introduce the class `Consumer`
- Consume data from Kafka topic

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import Consumer, KafkaError
import json
import qcutils

topic = ''
consumer_group = ''

assert len(topic) > 0, "In order to avoid conflicts during write operation, please name the topic as <surname>-topic"
assert len(consumer_group) > 0, "In order to avoid conflicts during write operation, please name the the consumer_group as <surname>-cg"


servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))

**Note**: in order to avoid conflicts during read operation, please name:
* the topic as `<surname>-topic`
* the consumer group as follow `<surname>-cg`

In [ ]:
consumerconf = {
        'bootstrap.servers': servers,
        'group.id': consumer_group,
        'auto.offset.reset': 'earliest'
    }

c = Consumer(consumerconf)

## Consumer

The main part of a typical Kafka consumer application is the `consume loop`

The application repeatedly calls the `poll(n)` method to retrieve records. If no message is available in n seconds, the poll return an empty element.

The `subscribe()` method specifies which topics should be polled.

## Write a simple consumer (without Decoding)

In [ ]:
c.subscribe([topic])                                             # Subscribe to topic

waiting = False

try:
    while True:
        msg = c.poll(1.0)                                        # retrieve records
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = msg.value()                                  # get the value of the message
            key = msg.key()                                      # get the key of the message
            print("\nConsumed record with key {} and value {}"
                  .format(key, value))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()                                                   # Leave group and commit final offsets

Worse than the Console Consumer....

## Consume with Decoding

### Using a text decoder

Let's try decoding the message

In [ ]:
c.subscribe([topic])

waiting = False

try:
    while True:
        msg = c.poll(1.0)
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = msg.value().decode("UTF-8")                 ## deconding the message value as UTF-8
            key = msg.key()
            print("\nConsumed record with key {} and value {}"
                  .format(key, value))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()

We can print strange char as in the Console Consumer

But the number is still blank on this side...

How can we deal with it?

### Using a integer decoder

We need to call the right decoding function for the differen data types

In [ ]:
c.subscribe([topic])

waiting = False

try:
    while True:
        msg = c.poll(1.0)
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = int.from_bytes(msg.value(), byteorder='big')  ## deconding the messagevalue as an int
            key = msg.key()
            print("\nConsumed record with key {} and value {}"
                  .format(key, value))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()

But this consumer does not work for String, float, etc...

**Are you sure that you want to manually call all the possible decoding function inside the Consumer?**

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.